<a href="https://colab.research.google.com/github/kartika-nair/CAPTCHA-Solver/blob/master/DistortedTextPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import string
from tqdm.notebook import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import multiprocessing as mp

import cv2
import imutils
import tqdm as tq
import pickle
import os.path
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from google.colab.patches import cv2_imshow


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**PROJECT DATA**

In [ ]:
cpu_count = mp.cpu_count()

IMAGES_FOLDER = "/content/drive/MyDrive/Colab Notebooks/DistortedText/Pytorch/Images"

EXTRACT_IMAGES = 0 #SET THIS TO TRUE TO GENERATE IMAGES READBLE BY CODE
NUMBER_OF_LETTERS = 4

EXTRACTED_FOLDER = "/content/drive/MyDrive/Colab Notebooks/DistortedText/Pytorch/EXTRACTED"

IMAGE_SIZE = 32


**EXTRACT IMAGES FROM CAPTCHA IMAGES**

In [ ]:
def extract_image(path = IMAGES_FOLDER, output = EXTRACTED_FOLDER):
    captcha_image_files = glob.glob(os.path.join(path, "*"))
    counts = {}
    for (i, captcha_image_file) in tq.tqdm(enumerate(captcha_image_files)):
        # print(f"[DEBUG]{i}: {os.path.basename(captcha_image_file)}")

        try:
            filename = os.path.basename(captcha_image_file)
            captcha_correct_text = os.path.splitext(filename)[0]

            image = cv2.imread(captcha_image_file)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)

            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

            contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            contours = contours[1] if imutils.is_cv3() else contours[0]

            letter_image_regions = []

            for contour in contours:
                (x, y, w, h) = cv2.boundingRect(contour)

                if w / h > 1.25:

                    half_width = int(w / 2)
                    letter_image_regions.append((x, y, half_width, h))
                    letter_image_regions.append((x + half_width, y, half_width, h))

                else:

                    letter_image_regions.append((x, y, w, h))

            if len(letter_image_regions) != NUMBER_OF_LETTERS:
                continue

            letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

            for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_correct_text):
                x, y, w, h = letter_bounding_box
                letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]

                save_path = os.path.join(output, "")

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            count = counts.get(letter_text, 1)
            p = os.path.join(save_path, "{}_{}.png".format(letter_text, str(count).zfill(6)))
            cv2.imwrite(p, letter_image)

            counts[letter_text] = count + 1
        except Exception as e:
            pass


In [ ]:
if EXTRACT_IMAGES:
    extract_image()

In [ ]:
def resize_to_fit(image, width = IMAGE_SIZE, height = IMAGE_SIZE):

    (h, w) = image.shape[:2]

    if w > h:
        image = imutils.resize(image, width=width)
    else:
        image = imutils.resize(image, height=height)

    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    return image

**CUSTOM DATASET CLASS**

In [ ]:
class CAPTCHADataset(Dataset):

    def __init__(self, images, data_dir = EXTRACTED_FOLDER):
        self.data_dir = data_dir
        self.images = images
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_fn = self.images[index]
        image = cv2.imread(f"{self.data_dir}/{image_fn}")
        # image = Image.open(image_fp).convert('RGB')
        image = resize_to_fit(image)
        image = self.transform(image)
        text = image_fn.split("_")[0]
        return image, text
    
    def transform(self, image):
        
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)


**ENCODE DATASET**

In [ ]:
images = os.listdir(EXTRACTED_FOLDER)
image_fns_train, image_fns_test = train_test_split(images, random_state=0)
image_ns = [image.split("_")[0] for image in images]
image_ns = "".join(image_ns)
letters = sorted(list(set(list(image_ns))))
vocabulary = ["-"] + letters
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {v:k for k,v in idx2char.items()}

**MODEL PARAMETERS**

In [ ]:
batch_size = 6
num_epochs = 50
lr = 0.001
log_interval = 100
gamma = 0.7
num_chars = len(char2idx)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**DEFINE TEST AND TRAIN LOADER**

In [ ]:
trainset = CAPTCHADataset(images = image_fns_train, data_dir =  EXTRACTED_FOLDER) 
testset = CAPTCHADataset(images = image_fns_test, data_dir = EXTRACTED_FOLDER)
train_loader = DataLoader(trainset, batch_size=batch_size, num_workers=cpu_count, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, num_workers=cpu_count, shuffle=False)

In [ ]:
image_batch, text_batch = iter(train_loader).next()
num_chars = len(char2idx)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(12544, 128)
        self.fc2 = nn.Linear(128, 32)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
modelTest = Net()
modelTest(image_batch)

tensor([[-3.5127, -3.6010, -3.1507, -3.5474, -3.4267, -3.3147, -3.2178, -3.3107,
         -3.3685, -3.5525, -3.4175, -3.4722, -3.6291, -3.4764, -3.4547, -3.3513,
         -3.3436, -3.3090, -3.7825, -3.5965, -3.6585, -3.7621, -3.4247, -3.6718,
         -3.2866, -3.5469, -3.4015, -3.5309, -3.5189, -3.4720, -3.4733, -3.6763],
        [-3.5990, -3.3793, -3.4121, -3.3929, -3.7313, -3.6481, -3.3620, -3.2458,
         -3.4808, -3.2707, -3.3491, -3.5272, -3.5792, -3.7399, -3.3155, -3.2990,
         -3.4335, -3.2495, -3.6324, -3.4623, -3.5613, -3.6476, -3.2703, -3.3847,
         -3.5120, -3.6429, -3.6646, -3.5096, -3.6509, -3.4801, -3.2837, -3.5329],
        [-3.5521, -3.4682, -3.1800, -3.6213, -3.6790, -3.3756, -3.4565, -3.4056,
         -3.6410, -3.4283, -3.4397, -3.2539, -3.7661, -3.5775, -3.3626, -3.5180,
         -3.3721, -3.1818, -3.6121, -3.5299, -3.6634, -3.5630, -3.2958, -3.6680,
         -3.3834, -3.4428, -3.4298, -3.3764, -3.6941, -3.5135, -3.4508, -3.3424],
        [-3.6230, -3.4442

In [ ]:
model = Net().to(device)

In [ ]:
def encode_text_batch(text_batch, device):
    
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.LongTensor(text_batch_targets_lens)
    
    text_batch_concat = "".join(text_batch)
    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.LongTensor(text_batch_targets)
    
    return text_batch_targets.to(device), text_batch_targets_lens.to(device)

In [ ]:
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
loss_func = nn.CrossEntropyLoss()

In [ ]:
def train(epoch, train_loader = train_loader, device = device, log_interval = log_interval, model = model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target
        optimizer.zero_grad()
        output = model(data)
        target, _ = encode_text_batch(text_batch, device)
        if (data.size()[0] != batch_size or (target.size()[0] >= num_chars or target.size()[0] <= 0)):
            continue
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({round(100. * batch_idx / len(train_loader), 2)}%)]\tLoss: {round(loss.item(), 6)}')

In [ ]:
def test(model = model, device = device, test_loader = test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target
            target, _ = encode_text_batch(text_batch, device)
            if (data.size()[0] != batch_size or (target.size()[0] >= num_chars or target.size()[0] <= 0)):
                continue
            output = model(data)
            test_loss += loss_func(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, num_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/7264 (0.0%)]	Loss: 3.511806
Train Epoch: 1 [600/7264 (8.26%)]	Loss: 3.218011
Train Epoch: 1 [1200/7264 (16.52%)]	Loss: 2.897994
Train Epoch: 1 [1800/7264 (24.77%)]	Loss: 2.255287
Train Epoch: 1 [2400/7264 (33.03%)]	Loss: 2.159617
Train Epoch: 1 [3000/7264 (41.29%)]	Loss: 1.834369
Train Epoch: 1 [3600/7264 (49.55%)]	Loss: 1.772397
Train Epoch: 1 [4200/7264 (57.8%)]	Loss: 1.897608
Train Epoch: 1 [4800/7264 (66.06%)]	Loss: 2.221812
Train Epoch: 1 [5400/7264 (74.32%)]	Loss: 2.273295
Train Epoch: 1 [6000/7264 (82.58%)]	Loss: 2.074849
Train Epoch: 1 [6600/7264 (90.83%)]	Loss: 2.128443
Train Epoch: 1 [7200/7264 (99.09%)]	Loss: 1.568285

Test set: Average loss: 0.2766, Accuracy: 806/2422 (33%)

Train Epoch: 2 [0/7264 (0.0%)]	Loss: 1.587813
Train Epoch: 2 [600/7264 (8.26%)]	Loss: 2.364811
Train Epoch: 2 [1200/7264 (16.52%)]	Loss: 1.884482
Train Epoch: 2 [1800/7264 (24.77%)]	Loss: 2.165469
Train Epoch: 2 [2400/7264 (33.03%)]	Loss: 1.343225
Train Epoch: 2 [3000/7264 (41.29%)]	Lo